In [505]:
from utils import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [506]:
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html
class MLP(nn.Module) :
    def __init__(self, in_dim, out_dim, inter_dim=28*28) :
        super().__init__()
        self.linear1 = nn.Linear(in_dim, inter_dim, bias=True)
        self.act1 = nn.Sigmoid()
        self.linear2 = nn.Linear(inter_dim, out_dim, bias=True)
        
    def forward(self, x) :
#         assert x.size()[2]*x.size()[3] == self.in_dim, "x.size()[2]*x.size()[3] should be {}".format(self.in_dim)
        z = self.linear1(x.view(-1, 28*28))
        z = self.act1(z)
        z = self.linear2(z)
        return z

In [507]:
STUDENT_model = MLP(28*28, 1)

In [508]:
# Please change "STUDENT_model" with your NN model class's instance name
total_params = sum(p.numel() for p in STUDENT_model.parameters())

print("Number of Parameters: ", total_params)


Number of Parameters:  616225


In [509]:
# Normalize data with mean=0.5, std=1.0
mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (1.0,))
])

# download path 정의
download_root = './MNIST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

In [510]:
'''
Choose two numbers and prepare datasets with two numbers.
'''
nb1 = 3
nb2 = 8

train_dataset.data = torch.cat([train_dataset.data[(train_dataset.targets == nb1)],train_dataset.data[(train_dataset.targets == nb2)]],dim=0)
train_dataset.targets = torch.cat([train_dataset.targets[(train_dataset.targets == nb1)],train_dataset.targets[(train_dataset.targets == nb2)]],dim=0)

train_dataset.targets[(train_dataset.targets == nb1)] = 0
train_dataset.targets[(train_dataset.targets == nb2)] = 1

'''
Do the same thing for the test, valid dataset
'''
valid_dataset.data = torch.cat([valid_dataset.data[(valid_dataset.targets == nb1)],valid_dataset.data[(valid_dataset.targets == nb2)]],dim=0)
valid_dataset.targets = torch.cat([valid_dataset.targets[(valid_dataset.targets == nb1)],valid_dataset.targets[(valid_dataset.targets == nb2)]],dim=0)

valid_dataset.targets[(valid_dataset.targets == nb1)] = 0
valid_dataset.targets[(valid_dataset.targets == nb2)] = 1

test_dataset.data = torch.cat([test_dataset.data[(test_dataset.targets == nb1)],test_dataset.data[(test_dataset.targets == nb2)]],dim=0)
test_dataset.targets = torch.cat([test_dataset.targets[(test_dataset.targets == nb1)],test_dataset.targets[(test_dataset.targets == nb2)]],dim=0)

test_dataset.targets[(test_dataset.targets == nb1)] = 0
test_dataset.targets[(test_dataset.targets == nb2)] = 1

In [511]:
# option 값 정의
batch_size = 8
dim = 28*28
learning_rate = 1e-3

train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [512]:
my_loss = nn.BCELoss()

In [513]:
optimizer = optim.SGD(STUDENT_model.parameters(), learning_rate)

for epoch in range(12):
    STUDENT_model.train()
    epoch_loss = 0
    
    for i, (data, target) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        output = STUDENT_model(data)
        
        output = output.reshape(-1)
        act = nn.Sigmoid()
        loss = my_loss(act(output), target.float())
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f'Epoch: {epoch} | Train Loss: {epoch_loss / len(train_loader)}')

Epoch: 0 | Train Loss: 0.676976250511941
Epoch: 1 | Train Loss: 0.6274155931813377
Epoch: 2 | Train Loss: 0.5684228537079489
Epoch: 3 | Train Loss: 0.5007039587273299
Epoch: 4 | Train Loss: 0.43316576549303387
Epoch: 5 | Train Loss: 0.37507672963019845
Epoch: 6 | Train Loss: 0.3292975744981075
Epoch: 7 | Train Loss: 0.29417729536407144
Epoch: 8 | Train Loss: 0.2670674399436714
Epoch: 9 | Train Loss: 0.2457712928457397
Epoch: 10 | Train Loss: 0.22888839028238533
Epoch: 11 | Train Loss: 0.21484421718074817


In [514]:
### Test for train_dataset
count = 0
for i in range(train_dataset.targets.size()[0]):
    image = train_dataset.data[i]
    label = train_dataset.targets[i]

    prediction = (STUDENT_model(image.float().view(-1, 28*28)) >= torch.FloatTensor([0.5]))
    if prediction == label:
        count += 1

print('total: {}, correctly matched: {}, accuracy: {:.2f}%\n'.format(train_dataset.targets.size()[0], count, count/train_dataset.targets.size()[0] * 100. ) )

total: 11982, correctly matched: 10967, accuracy: 91.53%



In [515]:
''' Test for test_dataset '''
count = 0
wrong_answer = []
STUDENT_model.eval()
for i in range(test_dataset.targets.size()[0]):
    image = test_dataset.data[i]
    label = test_dataset.targets[i]

    prediction = STUDENT_model.forward(image.float().view(-1, 28*28)) >= torch.FloatTensor([0.5])
    if prediction == label:
        count += 1
    else:
        wrong_answer += [i]

print('total: {}, correctly matched: {} accuracy: {:.2f}%\n'.format(test_dataset.targets.size()[0], count, count/test_dataset.targets.size()[0] * 100. ) )

total: 1984, correctly matched: 1848 accuracy: 93.15%



In [528]:
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html
class MultiClassMLP(nn.Module) :
    def __init__(self, in_dim, out_dim, inter_dim=24*24) :
        super().__init__()
        self.in_dim = in_dim
        self.linear1 = nn.Linear(in_dim, inter_dim, bias=True)
        self.act1 = nn.Sigmoid()
        self.linear2 = nn.Linear(inter_dim, inter_dim, bias=True)
        self.act2 = nn.Sigmoid()
        self.linear3 = nn.Linear(inter_dim, out_dim, bias=True)
        
    def forward(self, x) :
        # assert x.size()[2]*x.size()[3] == self.in_dim, "x.size()[2]*x.size()[3] should be {}".format(self.in_dim)
        x = x.reshape(-1, self.in_dim)
        z = self.linear1(x)
        z = self.act1(z)
        z = self.linear2(z)
        z = self.act2(z)
        z = self.linear3(z)
        return z

In [529]:
# MultiClass Classfication
STUDENT_model = MultiClassMLP(28*28, 10)

In [530]:
# Please change "STUDENT_model" with your NN model class's instance name
total_params = sum(p.numel() for p in STUDENT_model.parameters())

print("Number of Parameters: ", total_params)

Number of Parameters:  790282


In [531]:
train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

In [532]:
# option 값 정의
batch_size = 32
dim = 28*28
learning_rate = 1e-1

train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [533]:
my_loss = nn.CrossEntropyLoss()

print(my_loss(torch.Tensor([1, 2, 3]), torch.Tensor([4, 5, 6])))
print(my_loss(torch.Tensor([1, 2, 3]), torch.Tensor([4, 2, 3])))

tensor(19.1141)
tensor(13.6685)


In [534]:
optimizer = optim.SGD(STUDENT_model.parameters(), learning_rate)

for epoch in range(7):
    STUDENT_model.train()
    epoch_loss = 0
    
    for i, (data, target) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        output = STUDENT_model(data)
        
        loss = my_loss(output, target)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f'Epoch: {epoch} | Train Loss: {epoch_loss / len(train_loader)}')

Epoch: 0 | Train Loss: 1.4033473454634349
Epoch: 1 | Train Loss: 0.4381686601519585
Epoch: 2 | Train Loss: 0.3622001539846261
Epoch: 3 | Train Loss: 0.32194425857365133
Epoch: 4 | Train Loss: 0.2928644742747148
Epoch: 5 | Train Loss: 0.26796254447102547
Epoch: 6 | Train Loss: 0.24509243270655473


In [535]:
''' Test for train_dataset '''

image_size = 28*28
#image_size = 3*32*32 # for CIFAR dataset
STUDENT_model.eval()
batch_size = 32

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
count = 0
for image,label in train_loader :
    label = label
    prediction = torch.argmax(STUDENT_model(image.view(len(image),image_size)),dim=1)
    count += (prediction == label).sum()
        
print('Test for train_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(train_dataset.targets), count/len(train_dataset.targets) * 100. ) )    

''' Test for test_dataset '''
count = 0
wrong_answer = []
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
for image,label in test_loader :
    label = label
    prediction = torch.argmax(STUDENT_model(image.view(len(image),image_size)),dim=1)
    count += (prediction == label).sum()
    
#     else:
#         wrong_answer += [i]   
        
print('Test for test_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(test_dataset.targets), count/len(test_dataset.targets) * 100. ) )    

Test for train_dataset: correctly matched/total: 56187/60000 accuracy: 93.65%

Test for test_dataset: correctly matched/total: 9357/10000 accuracy: 93.57%

